# Step by step processing of PIScO images from fullframe PNGs to EcoTaxa Export per cruise

## 0.  Connect to docker container & import of libraries
Prerequesits
- Start the rocm_container if not already running.
- Start the jupyter server in the container (sudo docker exec -it rocm_container bash -c "jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root"): note the token/password
- Find the ip address of the rocm container (sudo docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' rocm_container)
- Connect to the python kernel in VS Code: Select other kernel... with IP addres and password/token from above.

In [1]:
!pip install numpy==2.2 pandas matplotlib scikit-image opencv-python umap-learn sqlalchemy tqdm standard-imghdr torch torchvision
import os
#os.chdir('/app/ImageProcessing/')
os.chdir('/home/veit/PIScO_dev/ImageProcessing/')

In [1]:
#### External modules
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0, 1"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import re
import datetime

from pandas.errors import EmptyDataError

from sqlalchemy import create_engine
from sqlalchemy import text

import inspect
from skimage import measure
from skimage.io import imread
import cv2
import umap
import pickle
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm
import logging
import shutil

#### Internal modules

from segmenter import run_segmenter
import analyze_profiles_seavision as ap



/home/veit/miniconda3/envs/pisco/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-27 15:41:09,238 - PyTorch version 2.6.0+rocm6.4.1.git1ded221d available.
2025-06-27 15:41:09,344 - Connected to the database


## 1.  Define the input images i.e. cruise folder


In [2]:
#Don't forget to mount the SMB share, in system terminal: sudo mount -t cifs //filer.geomar.de/projekte/ORTC-ST-PISCO /mnt/filer -o username=GEOM-svcPISCO_rw@geomar.de,password=2mZmhx-7GCGf 
# Select the base dir of the PIScO data for the cruise you want to process. Example: /mnt/filer/SO298/SO298-Logfiles_PISCO

### KOSMOS Peru 2020
# cruise = "PISCO_KOSMOS_2020_Peru"
# intermediate_result_dir = "/home/veit/PIScO_dev/Segmentation_results/PISCO_KOSMOS_2020_Peru/TempResults"
# cruise_base = f"/mnt/filer/PISCO_KOSMOS_2020_Peru/PIScO_Peru_source"
# ctd_dir= None
# #ctd_prefix = None
# log_directory = None

### HE570
# cruise = "HE570"
# cruise_base = "/mnt/filer/HE570/HE570-PISCO-Profiles"
# intermediate_result_dir = f"/home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults"
# ctd_dir = None
# #ctd_prefix = "met_202_1_"
# log_directory = None

### M181
# cruise = "M181"
# cruise_base = "/home/veit/PIScO_dev/Segmentation_results/M181"
# intermediate_result_dir = f"/home/veit/PIScO_dev/Segmentation_results/M181"
# ctd_dir = "/home/veit/Downloads/CTD_preliminary_calibrated"
# ctd_prefix = "met_181_1_"
# log_directory = "/home/veit/Downloads/Templog"
# pressure_unit = "bar"  # or "dbar"

### SO_298
# cruise = "SO298"
# cruise_base = "/mnt/filer/SO298/SO298-PISCO-Profiles"
# # #cruise_base = "/home/veit/PIScO_dev/usb_mount/SO298-PNG"
# intermediate_result_dir = "/home/veit/PIScO_dev/Segmentation_results/SO298/SO298-PISCO-Profiles/TempResults"
# ctd_dir = "/mnt/filer/SO298/SO298-CTD_UVP_ETC/SO298-CTD/calibrated/"
# ctd_prefix = "son_298_1_"
# log_directory = "/mnt/filer/SO298/SO298-Logfiles_PISCO/Templog"

### MSM_126
# cruise = "MSM126"
# cruise_base = "/mnt/filer/MSM126/MSM126-PISCO-Profiles"
# intermediate_result_dir = f"/home/veit/PIScO_dev/Segmentation_results/MSM126/MSM126-PISCO-Profiles/TempResults/"
# ctd_dir = "/mnt/filer/MSM126/MSM126-Data-UVP-CTD-ADCP/CTD/msm_126_1_ctd"
# ctd_prefix = "msm_126_1_"
# log_directory = "/mnt/filer/MSM126/MSM126-PISCO_Logfilesetc/Logfiles"

## M202
# cruise = "M202"
# cruise_base = "/mnt/filer/M202/M202-PISCO-Profiles"
# intermediate_result_dir = f"/home/veit/PIScO_dev/Segmentation_results/M202/M202-PISCO-Profiles/TempResults"
# ctd_dir = "/mnt/filer/M202/M202-external-Data/M202-CTD/met_202_1_ctd/met_202_1_ctd"
# ctd_prefix = "met_202_1_"
# log_directory = "/mnt/filer/M202/M202-Pisco-Logfiles/Logfiles"

### SO_308
cruise = "SO308"
cruise_base = "/mnt/filer/SO308/SO308-PISCO-Profiles"
intermediate_result_dir = "/home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults"
ctd_dir = "/mnt/filer/SO308/ADCP-ETC/CTD/SO308_ctd_files"
ctd_prefix = "son_308_1_"
log_directory = "/mnt/filer/SO308/PISCO-Logfiles/PISCO-LOGFILES/Logfiles"
pressure_unit = "dbar"  # or "bar"




## 2.  Main loop
### 2.1 Segmentation: create crops and corresponding unrefined metadata 


In [ ]:
for profile in os.listdir(cruise_base):
        print(profile)
        profile_path = os.path.join(cruise_base, profile)
        if os.path.isdir(profile_path):
                img_folder = os.path.join(profile_path, profile+"_Images-PNG")
                #img_folder = profile_path
                results_folder = os.path.join(intermediate_result_dir, profile, profile+"_Results")
                if not os.path.exists(img_folder):
                        print(f"Image folder {img_folder} does not exist, skipping profile {profile}.")
                        continue
                if not os.path.exists(results_folder):
                        os.makedirs(results_folder)
                run_segmenter(img_folder, results_folder, deconvolution=False)

### 2.2 Postprocessing: filtering, metadata curation, EcoTaxa export

In [3]:
def clean_ecotaxa_folders(base_folder, remove_all=False):
    """
    Remove EcoTaxa folders and their contents from the given base folder.
    
    Args:
        base_folder (str): Path to the base directory to clean
        remove_all (bool): If True, removes the entire EcoTaxa folder structure.
                         If False, only removes unzipped upload folders but keeps zips.
    """
    for root, dirs, files in os.walk(base_folder):
        if "EcoTaxa" in dirs:
            ecotaxa_path = os.path.join(root, "EcoTaxa")
            
            if remove_all:
                # Remove the entire EcoTaxa folder
                print(f"Removing entire EcoTaxa folder: {ecotaxa_path}")
                shutil.rmtree(ecotaxa_path)
            else:
                # Only remove unzipped upload folders
                for item in os.listdir(ecotaxa_path):
                    item_path = os.path.join(ecotaxa_path, item)
                    if os.path.isdir(item_path) and ("upload" in item or "crops" in item):
                        print(f"Removing unzipped folder: {item_path}")
                        shutil.rmtree(item_path)
                
                # Remove empty EcoTaxa folder if no files remain
                if not os.listdir(ecotaxa_path):
                    print(f"Removing empty EcoTaxa folder: {ecotaxa_path}")
                    shutil.rmtree(ecotaxa_path)
        elif "crops.zip" or "deconv_" in files:
            # If only crops.zip is present, remove it
            zip_path = os.path.join(root, "crops.zip")
            if os.path.exists(zip_path):
                if remove_all:
                    print(f"Removing crops.zip file: {zip_path}")
                    os.remove(zip_path)
            else:
                zip_path = os.path.join(root, "deconv_crops.zip")
                if os.path.exists(zip_path):    
                    if remove_all:
                        print(f"Removing deconv_crops.zip file: {zip_path}")
                        os.remove(zip_path)
            

# Usage examples:
# Remove only unzipped folders:
clean_ecotaxa_folders(intermediate_result_dir, remove_all=True)

Removing crops.zip file: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-28-1_PISCO2_20241124-2155/SO308_1-28-1_PISCO2_20241124-2155_Results/crops.zip
Removing crops.zip file: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-2-5_PISCO2_20241105-0723/SO308_1-2-5_PISCO2_20241105-0723_Results/crops.zip
Removing crops.zip file: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-29-1_PISCO2_20241125-1314/SO308_1-29-1_PISCO2_20241125-1314_Results/crops.zip
Removing crops.zip file: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-29-3_PISCO2_20241125-1829/SO308_1-29-3_PISCO2_20241125-1829_Results/crops.zip
Removing crops.zip file: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-38-3_PISCO2_20241208-0748/SO308_1-38-3_PISCO2_20241208-0748_Results/crops.zip
Removing crops.zip file: /home/veit/PIScO_

In [4]:
predict_ViT = False  # Set to True if you want to run the ViT predictions, False otherwise

for profile in os.listdir(cruise_base):
    print(profile)
    profile_path = os.path.join(cruise_base, profile)
    
    if os.path.isdir(profile_path):
        # Some variables
        if cruise == "M181":
            results_folder = os.path.join(intermediate_result_dir, profile)# M181
        else:
            results_folder = os.path.join(intermediate_result_dir, profile, profile+"_Results")
       
        # if "crops_excluded" in os.listdir(results_folder):
        #     print(f"Profile {profile} already processed. Skipping.")
        #     continue
        if cruise == "HE570":
            profile_id = profile.split('_')[1]+profile.split('_')[2]  # HE570 profile ID is the second and the third part of the profile name
        elif cruise == "M181":
            profile_id = profile.split('_')[0].split('-')[1]+'-'+profile.split('_')[0].split('-')[2]  # M181 Extract the profile ID from the profile name
        else:
            profile_id = profile.split('_')[1]  # Extract the profile ID from the profile name
        
        profile_data_dir = os.path.join(results_folder, 'Data')
        ecotaxa_dir = os.path.join(results_folder, 'EcoTaxa')

        if os.path.exists(ecotaxa_dir):
            print(f"EcoTaxa directory {ecotaxa_dir} already exists. Skipping profile {profile}.")
            continue

        # Load the results of the segmenter
        # and create a DataFrame with the particle data
        df = ap.gen_crop_df(profile_data_dir, small=False, size_filter=0, cruise=cruise, pressure_unit=pressure_unit)
        print(len(df.index), 'particles found.')
        df['fullframe_path'] = df['full_path'].apply(ap.modify_full_path)
        #
        threshold_ratio = 0.005  # Define the aspect ratio threshold
        print(len(df)-len(df[df['w'] >= threshold_ratio * df['h']]), 'crops removed due to aspect ratio')
        df = df[df['w'] >= threshold_ratio * df['h']]

        # Add ctd data to the DataFrame
        # Get CTD profile ID
        if ctd_dir is not None and os.path.exists(ctd_dir):
            if cruise == "M181":
                ctd_id = profile.split('_')[1].split('-')[1]   # M181 profile ID is the second part of the profile name
            else:
                ctd_id = ap.get_ctd_profile_id(cruise_base, profile)           
            if ctd_id:
                ctd_file = os.path.join(ctd_dir, f'{ctd_prefix}{ctd_id}.ctd')
                print(f'adding ctd data...{ctd_file}')
                # Check if the CTD file exists
                if not os.path.exists(ctd_file):
                    print(f"CTD file {ctd_file} does not exist, skipping CTD data for profile {profile}.")
                    continue
                df = ap.add_ctd_data(ctd_file, df)
            else:
                print(f"Skipping CTD data for {profile}")
                continue

        # Predict on deconvolved images  
        if predict_ViT:
            img_dir = os.path.join(results_folder, "Deconv_crops")  
            ds_pisco = ap.load_unclassified_images(img_dir)

            # Apply the transform to the dataset
            ds_pisco_trans = ds_pisco.with_transform(ap.process_batch)

            # Define model name and path to the saved model
            model_dir = '/home/veit/PIScO_dev/ViT_custom_size_sensitive/best_model'

            # Get predictions on the dataset in batches
            # Using entropy-based OOD detection
            filenames, predictions, probabilities, entropy_scores, ood_flags = ap.get_predictions_with_entropy_ood(
                ds_pisco_trans, 
                model_dir, 
                entropy_threshold=0.9,  # Tune this threshold
                temperature=1.5,        # Tune this parameter
                batch_size=64
            )
            # Create DataFrame with OOD detection results
            prediction_df = pd.DataFrame({
                'filename': filenames,
                'is_ood': ood_flags,
                'entropy': entropy_scores,
                'top1': [pred[0] for pred in predictions],
                'top2': [pred[1] for pred in predictions],
                'top3': [pred[2] for pred in predictions],
                'top4': [pred[3] for pred in predictions],
                'top5': [pred[4] for pred in predictions],
                'prob1': [prob[0] for prob in probabilities],
                'prob2': [prob[1] for prob in probabilities],
                'prob3': [prob[2] for prob in probabilities],
                'prob4': [prob[3] for prob in probabilities],
                'prob5': [prob[4] for prob in probabilities]
            })
            prediction_df['filename'] = prediction_df['filename'].apply(os.path.basename)
            result_path = os.path.join(results_folder, 'ViT_predictions.csv')
            prediction_df.to_csv(result_path, index=False)
            print("Predictions saved!")

            # Merge predictions with the main DataFrame
            merged_df = pd.merge(
                df,
                prediction_df,
                on='filename',
                how='left',
                validate='1:1'  # Ensures one-to-one merge
            )
    
            # Verify merge success
            print("Number of null values in top1:", merged_df['top1'].isna().sum())
            df = merged_df.copy()   

        # Add log data to the DataFrame
        if log_directory is not None:                
            print('adding log info...')
            timestamp = profile.split('_')[-1]
            # Convert timestamp to datetime object
            date_time_obj = datetime.datetime.strptime(timestamp, '%Y%m%d-%H%M')
            min_diff = datetime.timedelta(days=365*1000)  # initialize with a big time difference
            closest_file = None

            # Iterate over all files in the directory
            for filename in os.listdir(log_directory):
                # Check if filename is a Templog
                if '__Templog.txt' in filename:
                    # Extract timestamp from filename and convert to datetime object
                    file_timestamp = filename[:16]
                    file_datetime = datetime.datetime.strptime(file_timestamp, '%Y%m%d_%Hh_%Mm')

                    # Calculate time difference
                    diff = abs(date_time_obj - file_datetime)

                    # If this file is closer, update min_diff and closest_file
                    if diff < min_diff:
                        min_diff = diff
                        closest_file = filename

            if closest_file is None:
                print("Logfile not found")
            else:
                file_path = os.path.join(log_directory, closest_file)
                file_size = os.path.getsize(file_path)  # Get file size in bytes
                print(f"Closest logfile: {closest_file}, Size: {file_size} bytes")
            
            # Read the log file and parse the relevant data

            df_log = ap.create_log_df(file_path)

            # Match the data with the profile dataframe
            df.drop(['TT_x', 'T1_x', 'T2_x', 'TH_x', 'restart_x', 'relock_x', 'Time_log_x', 'TT_y', 'T1_y', 'T2_y', 'TH_y', 'restart_y', 'relock_y', 'Time_log_y', 'TT', 'T1', 'T2', 'TH', 'restart', 'relock', 'Time_log'], axis=1, inplace=True, errors='ignore')
            # Convert the timestamps in both dataframes to datetime format
            df['timestamp'] = df['date-time']

            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d-%H%M%S%f')

            # Sort the dataframes by the timestamp
            df = df.sort_values('timestamp')
            df_log = df_log.sort_values('timestamp')

            # Use merge_asof to merge the two dataframes, finding the nearest match on the timestamp
            df_combined = pd.merge_asof(df, df_log, left_on='timestamp', right_on='timestamp', direction='backward')
            df_combined.drop('timestamp', axis=1, inplace=True)

            # First check which columns are available
            required_columns = ['date-time', 'pressure [dbar]', 'img_id']  # These are the minimum required columns

            # Add optional columns if they exist
            optional_columns = ['depth [m]', 'temperature', 'interpolated_s', 'interpolated_t', 
                            'interpolated_o', 'interpolated_z_factor', 'restart', 'relock', 'TAG_event']

            # Get list of columns that actually exist in df_combined
            existing_columns = [col for col in required_columns + optional_columns if col in df_combined.columns]

            #Add particle count based filter for filtering out images that are potentially obscured by schlieren or bubbles
            df_unique = df_combined[existing_columns].drop_duplicates()
            df_count = df_combined.groupby('date-time').size().reset_index(name='count')
            df_unique = df_unique.merge(df_count, on='date-time', how='left')
            df_unique = df_unique.sort_values('pressure [dbar]')
            df_unique['part_based_filter'] = df_unique['count'].apply(lambda x: 0 if x < (df_unique['count'].mean() + 5*df_unique['count'].std()) else 1)
            df = df_combined.merge(df_unique[['date-time', 'part_based_filter']], on='date-time', how='left')
        
        else:
            print('No log directory provided, skipping log data.')
            # Use merge_asof to merge the two dataframes, finding the nearest match on the timestamp
            df_combined = df.copy()

            # First check which columns are available
            required_columns = ['date-time', 'pressure [dbar]', 'img_id']  # These are the minimum required columns

            # Add optional columns if they exist
            optional_columns = ['depth [m]', 'temperature', 'interpolated_s', 'interpolated_t', 
                            'interpolated_o', 'interpolated_z_factor', 'restart', 'relock', 'TAG_event']

            # Get list of columns that actually exist in df_combined
            existing_columns = [col for col in required_columns + optional_columns if col in df_combined.columns]

            #Add particle count based filter for filtering out images that are potentially obscured by schlieren or bubbles
            df_unique = df_combined[['date-time', 'pressure [dbar]', 'img_id']].drop_duplicates()
            df_count = df_combined.groupby('date-time').size().reset_index(name='count')
            df_unique = df_unique.merge(df_count, on='date-time', how='left')
            df_unique = df_unique.sort_values('pressure [dbar]')
            df_unique['part_based_filter'] = df_unique['count'].apply(lambda x: 0 if x < (df_unique['count'].mean() + 5*df_unique['count'].std()) else 1)
            df = df_combined.merge(df_unique[['date-time', 'part_based_filter']], on='date-time', how='left')
        
        print(len(df.index), 'particles after adding log data.')


        ## Process for exporting to EcoTaxa
        # Add object ID column
        df['object_id'] = profile_id + df['img_id'].astype(str) + '_' + df['index'].astype(str) 

        # Get latitude and longitude from yaml file
        # Assuming your YAML file is in the same directory as your profile
        if cruise == "HE570":
            yaml_file = os.path.join(profile_path, profile + ".yaml")
            #split date-time
            df[['date', 'time']] = df['date-time'].str.split('-', expand=True)
        
        elif cruise == "M181":
            latitude, longitude = ap.extract_lat_lon_from_profile(profile)
            # Split datetime into date and time based on the actual format
            # First extract the date part (before the '-') and time part (after the '-')
            df['date'] = df['date-time'].str.split('-').str[0]
            df['time'] = df['date-time'].str.split('-').str[1]
            yaml_file = False  # No YAML file for M181, using extracted lat/lon directly
        
        else:
            # For other cruises, the YAML file is in a subdirectory
            yaml_file = os.path.join(profile_path, profile + "_Metadata", profile + ".yaml")
            #split date-time
            df[['date', 'time']] = df['date-time'].str.split('-', expand=True)
        
        if yaml_file:
            latitude, longitude = ap.extract_coords_from_yaml(yaml_file)

        # Add the columns to your DataFrame and fill all rows with the same values
        df['lat'] = latitude
        df['lon'] = longitude

        ## Filter crops based on aspect ratio and artifacts (TAG events and schlieren)
        
        if "TAG_event" in df.columns:
            print(len(df)-len(df[df['TAG_event'] == 0]), 'crops removed due to TAG events')
            df = df[df['TAG_event'] == 0]

        # Filter out crops with part_based_filter == 1   
        print(len(df)-len(df[df['part_based_filter'] == 0]), 'crops removed due to particle count based filter')
        df = df[df['part_based_filter'] == 0]  
        
        print(len(df), 'crops remaining after filtering')

        # Remove duplicates in the 'full_path' column, keeping the first occurrence
        df = df.drop_duplicates(subset='full_path', keep='first')
        df = df.sort_values(by='object_id')
        df.reset_index(drop=True, inplace=True)
        print(len(df), 'crops after removing duplicates')

        # Prepare EcoTaxa metadata
        #print(df.head())
        df_ET = ap.rename_for_ecotaxa(df, sample_profile_id=profile, predicted=predict_ViT).copy()

        
        if predict_ViT:
            # Add debugging and fix the MultiIndex column access
            # print("DataFrame info:")
            # print(df_ET.info())
            # print("\nIndex type:", type(df_ET.index))
            # print("\nColumns:", df_ET.columns)
            # Access the column using the full MultiIndex specification
            annotation_col = ('object_annotation_category', '[t]')
            # print("\nSample of object_annotation_category:")
            # print(df_ET[annotation_col].head())

            # Filter using the proper column reference
            df_ET = df_ET[df_ET[annotation_col].notna()]

            # Verify the result
            # print("\nShape after filtering:", df_ET.shape)
            # print("Remaining NaN values:", df_ET[annotation_col].isna().sum())  
            #   
        results_folder = os.path.join(results_folder, 'EcoTaxa')
        if not os.path.exists(results_folder):
            os.makedirs(results_folder)  # Create directory if it doesn't exist
            
        if not df_ET.empty:
            ap.create_ecotaxa_zips(output_folder=results_folder, df=df_ET, profile_name=profile, 
                                max_zip_size_mb=500, compression_ratio=1.0, copy_images=True, 
                                add_scale_bar_to_deconv=False, pixel_resolution=23, scale_length_mm=1)
        else:
            print(f"No data to process for profile {profile}")

        #ap.create_ecotaxa_zips(output_folder=results_folder, df=df_ET, profile_name=profile, max_zip_size_mb=500, compression_ratio=1.0, copy_images=True, add_scale_bar_to_deconv=False, pixel_resolution=23, scale_length_mm=1)

        # Save the DataFrame to a CSV file
        df.to_csv(os.path.join(results_folder, f'{profile}_crops_metadata.csv'), index=False)

        # Create plots
        # plot_path = os.path.join(results_folder, 'Plots')
        # if not os.path.exists(plot_path):
        #     os.makedirs(plot_path)
        # ap.plot_histogram(df, plot_path)
        # ap.plot_position_hist(df, plot_path)
        # ap.plot_2d_histogram(df, plot_path)
        # press_min = df['object_pressure'].min()-10
        # depth_bin_size = 1
        # _, pivoted_df = ap.populate_esd_bins_pressure(df,  depth_bin_size=depth_bin_size, esd_bins=np.array([0,125,250,500,1000,100000]))
        # ap.plot_particle_dist(pivoted_df, folder, plot_path, depth_bin_size=depth_bin_size, preliminary=True, depth_min=press_min)
        # ap.plot_particle_dist(pivoted_df, folder, plot_path, depth_bin_size=depth_bin_size, preliminary=True, depth_min=press_min, maximum_y_value=500)
        # ap.plot_ctd_data(df, profile, plot_path)
        # ap.plot_ctd_data(df, profile, plot_path, maximum_y_value=500)




SO308_1-37-4_PISCO2_20241207-1021
EcoTaxa directory /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-37-4_PISCO2_20241207-1021/SO308_1-37-4_PISCO2_20241207-1021_Results/EcoTaxa already exists. Skipping profile SO308_1-37-4_PISCO2_20241207-1021.
SO308_1-28-1_PISCO2_20241124-2155
EcoTaxa directory /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-28-1_PISCO2_20241124-2155/SO308_1-28-1_PISCO2_20241124-2155_Results/EcoTaxa already exists. Skipping profile SO308_1-28-1_PISCO2_20241124-2155.
SO308_1-38-3_PISCO2_20241208-0748
EcoTaxa directory /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-38-3_PISCO2_20241208-0748/SO308_1-38-3_PISCO2_20241208-0748_Results/EcoTaxa already exists. Skipping profile SO308_1-38-3_PISCO2_20241208-0748.
SO308_1-20-1_PISCO2_20241116-1129
EcoTaxa directory /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-20-1

100%|██████████| 37714/37714 [01:23<00:00, 454.14it/s]


24994


2025-06-27 15:49:44,290 - crop_df created


   cruise dship_id instrument  pressure [dbar]          date-time  index  \
3   SO308      0-0     PISCO2             9.62  20241104-06185090      2   
9   SO308      0-0     PISCO2             9.62  20241104-06185090      8   
26  SO308      0-0     PISCO2             9.62  20241104-06202990      5   
29  SO308      0-0     PISCO2             9.62  20241104-06202990      8   
99  SO308      0-0     PISCO2             9.63  20241104-06154350     64   

       lat      lon  temperature  img_id  ... object_boundbox_area  \
3   26.14S  034.71E        21.95       2  ...               3864.0   
9   26.14S  034.71E        21.95       2  ...               4100.0   
26  26.14S  034.71E        22.18       5  ...               3240.0   
29  26.14S  034.71E        22.18       5  ...              19352.0   
99  26.14S  034.71E        21.70       9  ...              15444.0   

    object_eccentricity  object_equivalent_diameter  object_euler_nr  \
3              0.987445                   30.17227

/home/veit/PIScO_dev/ImageProcessing/analyze_profiles_seavision.py:331: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  ctd_df = pd.read_csv(ctd_data_loc,
2025-06-27 15:49:52,949 - CTD data added to crop_df
2025-06-27 15:49:53,033 - Reading log file: /mnt/filer/SO308/PISCO-Logfiles/PISCO-LOGFILES/Logfiles/20241104_06h_09m__Templog.txt
2025-06-27 15:49:53,036 - Unexpected TT line format: b'TT_20.69_T1_20.69_T2_20.31_C1_27.25_C2_26.81_TH_20.69_PWM_0_ki_0.00_kp_70.0k_.'. Parsing TT data: ['TT', '20.69', 'T1', '20.69', 'T2', '20.31', 'C1', '27.25', 'C2', '26.81', 'TH', '20.69', 'PWM', '0', 'ki', '0.00', 'kp', '70.0k', '.'], number of values: 19
2025-06-27 15:49:53,036 - skipping line in file /mnt/filer/SO308/PISCO-Logfiles/PISCO-LOGFILES/Logfiles/20241104_06h_09m__Templog.txt: b'TT_20.69_T1_20.69_T2_20.31_C1_27.25_C2_26.81_TH_20.69_PWM_0_ki_0.00_kp_70.0k_.'


adding log info...
Closest logfile: 20241104_06h_09m__Templog.txt, Size: 478595 bytes


/home/veit/PIScO_dev/ImageProcessing/analyze_profiles_seavision.py:1057: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['restart', 'relock']] = df[['restart', 'relock']].fillna(False).astype(bool)


92248 particles after adding log data.
5055 crops removed due to TAG events
7711 crops removed due to particle count based filter
79482 crops remaining after filtering
78881 crops after removing duplicates
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_0-0_PISCO2_20241104-0610/SO308_0-0_PISCO2_20241104-0610_Results/Crops
Created zip file from source folder: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_0-0_PISCO2_20241104-0610/SO308_0-0_PISCO2_20241104-0610_Results/EcoTaxa/crops.zip
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_0-0_PISCO2_20241104-0610/SO308_0-0_PISCO2_20241104-0610_Results/Deconv_crops
Created zip file from source folder: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_0-0_PISCO2_20241104-0610/SO308_0-0_PISCO2_20241104-0610_Results/EcoTaxa/deconv_crops.zip
SO308_

In [28]:
# Add these diagnostic lines before filtering
print("DataFrame info:")
print(df_ET.info())
print("\nIndex type:", type(df_ET.index))
print("\nColumns:", df_ET.columns)
print("\nSample of object_annotation_category:")
print(df_ET['object_annotation_category'].head())

# If it is a MultiIndex DataFrame, try this modified approach:
if isinstance(df_ET.index, pd.MultiIndex):
    # Reset the index to make it a regular DataFrame
    df_ET = df_ET.reset_index()
    
# Then apply the filter
df_ET = df_ET[df_ET['object_annotation_category'].notna()]

# Verify the result
print("\nShape after filtering:", df_ET.shape)
print("Remaining NaN values:", df_ET['object_annotation_category'].isna().sum())

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63312 entries, 0 to 63311
Data columns (total 53 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   (object_pressure, [f])               0 non-null      float64
 1   (object_depth_min, [f])              0 non-null      float64
 2   (img_file_name, [t])                 63298 non-null  object 
 3   (object_area, [f])                   0 non-null      float64
 4   (object_width, [f])                  0 non-null      float64
 5   (object_height, [f])                 0 non-null      float64
 6   (object_bound_box_w, [f])            0 non-null      float64
 7   (object_bound_box_h, [f])            0 non-null      float64
 8   (object_circularity, [f])            0 non-null      float64
 9   (object_area_exc, [f])               0 non-null      float64
 10  (object_area_rprops, [f])            0 non-null      float64
 11  (object_%are

In [ ]:
### KOSMOS Logic##########
# for profile in os.listdir(cruise_base):
#     print(profile)
#     profile_path = os.path.join(cruise_base, profile)
#     if os.path.isdir(profile_path):                
#             for mesokosm in os.listdir(profile_path):
#                     print(mesokosm)
#                     mesokosm_path = os.path.join(profile_path, mesokosm)
#                     if os.path.isdir(mesokosm_path):
#                             img_folder = mesokosm_path
#                             results_folder = os.path.join(intermediate_result_dir, profile, mesokosm+"_Results")
#                             profile_id = profile+'_'+mesokosm
#                             profile_data_dir = os.path.join(results_folder, 'Data')

#                             # Load the results of the segmenter
#                             # and create a DataFrame with the particle data
#                             df = ap.gen_crop_df(profile_data_dir, small=False, size_filter=0, cruise=cruise, pressure_unit='bar')
#                             print(len(df.index), 'particles found.')
#                             df['fullframe_path'] = df['full_path'].apply(ap.modify_full_path)

#                             # Add ctd data to the DataFrame
#                             # Get CTD profile ID
#                             if ctd_dir is not None and os.path.exists(ctd_dir):
#                                 ctd_id = ap.get_ctd_profile_id(cruise_base, profile)
#                                 if ctd_id:
#                                     ctd_file = os.path.join(ctd_dir, f'{ctd_prefix}{ctd_id}.ctd')
#                                     print('adding ctd data...')
#                                     df = ap.add_ctd_data(ctd_file, df)
#                                 else:
#                                     print(f"Skipping CTD data for {profile}")
#                                     continue

#                             # Predict on deconvolved images                
#                             prediction_file = os.path.join(results_folder, 'ViT_predictions.csv')  # Example path
#                             if os.path.exists(prediction_file):
#                                 print('Adding predictions...')
#                                 prediction_df = pd.read_csv(prediction_file)
#                                 df = ap.add_prediction(df, prediction_df)

#                             # Add log data to the DataFrame
#                             if log_directory is not None:                
#                                 print('adding log info...')
#                                 timestamp = profile.split('_')[-1]
#                                 # Convert timestamp to datetime object
#                                 date_time_obj = datetime.datetime.strptime(timestamp, '%Y%m%d-%H%M')
#                                 min_diff = datetime.timedelta(days=365*1000)  # initialize with a big time difference
#                                 closest_file = None

#                                 # Iterate over all files in the directory
#                                 for filename in os.listdir(log_directory):
#                                     # Check if filename is a Templog
#                                     if '__Templog.txt' in filename:
#                                         # Extract timestamp from filename and convert to datetime object
#                                         file_timestamp = filename[:16]
#                                         file_datetime = datetime.datetime.strptime(file_timestamp, '%Y%m%d_%Hh_%Mm')

#                                         # Calculate time difference
#                                         diff = abs(date_time_obj - file_datetime)

#                                         # If this file is closer, update min_diff and closest_file
#                                         if diff < min_diff:
#                                             min_diff = diff
#                                             closest_file = filename

#                                 if closest_file is None:
#                                     print("Logfile not found")
#                                 else:
#                                     file_path = os.path.join(log_directory, closest_file)
#                                     file_size = os.path.getsize(file_path)  # Get file size in bytes
#                                     print(f"Closest logfile: {closest_file}, Size: {file_size} bytes")
                                
#                                 # Read the log file and parse the relevant data

#                                 df_log = ap.create_log_df(file_path)

#                                 # Match the data with the profile dataframe
#                                 df.drop(['TT_x', 'T1_x', 'T2_x', 'TH_x', 'restart_x', 'relock_x', 'Time_log_x', 'TT_y', 'T1_y', 'T2_y', 'TH_y', 'restart_y', 'relock_y', 'Time_log_y', 'TT', 'T1', 'T2', 'TH', 'restart', 'relock', 'Time_log'], axis=1, inplace=True, errors='ignore')
#                                 # Convert the timestamps in both dataframes to datetime format
#                                 df['timestamp'] = df['date-time']

#                                 df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d-%H%M%S%f')

#                                 # Sort the dataframes by the timestamp
#                                 df = df.sort_values('timestamp')
#                                 df_log = df_log.sort_values('timestamp')

#                                 # Use merge_asof to merge the two dataframes, finding the nearest match on the timestamp
#                                 df_combined = pd.merge_asof(df, df_log, left_on='timestamp', right_on='timestamp', direction='backward')
#                                 df_combined.drop('timestamp', axis=1, inplace=True)

#                                 #Add particle count based filter for filtering out images that are potentially obscured by schlieren or bubbles
#                                 df_unique = df_combined[['date-time', 'pressure [dbar]', 'depth [m]', 'img_id','temperature','interpolated_s','interpolated_t','interpolated_o','interpolated_z_factor','restart','relock','TAG_event']].drop_duplicates()
#                                 df_count = df_combined.groupby('date-time').size().reset_index(name='count')
#                                 df_unique = df_unique.merge(df_count, on='date-time', how='left')
#                                 df_unique = df_unique.sort_values('pressure [dbar]')
#                                 df_unique['part_based_filter'] = df_unique['count'].apply(lambda x: 0 if x < (df_unique['count'].mean() + 5*df_unique['count'].std()) else 1)
#                                 df = df_combined.merge(df_unique[['date-time', 'part_based_filter']], on='date-time', how='left')
                            
#                             else:
#                                 print('No log directory provided, skipping log data.')
#                                 # Use merge_asof to merge the two dataframes, finding the nearest match on the timestamp
#                                 df_combined = df.copy()

#                                 #Add particle count based filter for filtering out images that are potentially obscured by schlieren or bubbles
#                                 df_unique = df_combined[['date-time', 'pressure [dbar]', 'img_id']].drop_duplicates()
#                                 df_count = df_combined.groupby('date-time').size().reset_index(name='count')
#                                 df_unique = df_unique.merge(df_count, on='date-time', how='left')
#                                 df_unique = df_unique.sort_values('pressure [dbar]')
#                                 df_unique['part_based_filter'] = df_unique['count'].apply(lambda x: 0 if x < (df_unique['count'].mean() + 5*df_unique['count'].std()) else 1)
#                                 df = df_combined.merge(df_unique[['date-time', 'part_based_filter']], on='date-time', how='left')


#                             ## Process for exporting to EcoTaxa
#                             # Add object ID column
#                             df['object_id'] = profile_id + df['img_id'].astype(str) + '_' + df['index'].astype(str) 

#                             #split date-time
#                             df[['date', 'time']] = df['date-time'].str.split('-', expand=True)

#                             ## Filter crops based on aspect ratio and artifacts (TAG events and schlieren)
#                             threshold_ratio = 0.005  # Define the aspect ratio threshold
#                             print(len(df)-len(df[df['w'] >= threshold_ratio * df['h']]), 'crops removed due to aspect ratio')
#                             df = df[df['w'] >= threshold_ratio * df['h']]
#                             if "TAG_event" in df.columns:
#                                 print(len(df)-len(df[df['TAG_event'] == 0]), 'crops removed due to TAG events')
#                                 df = df[df['TAG_event'] == 0]

#                             # Filter out crops with part_based_filter == 1   
#                             df = df[df['part_based_filter'] == 0]  

#                             # Remove duplicates in the 'full_path' column, keeping the first occurrence
#                             df = df.drop_duplicates(subset='full_path', keep='first')
#                             df = df.sort_values(by='object_id')
#                             df.reset_index(drop=True, inplace=True)
                            
#                             crops_zip = os.path.join(results_folder, "crops.zip")
#                             deconv_zip = os.path.join(results_folder, "deconv_crops.zip")
                            
#                             # # Check if both zip files already exist
#                             # if os.path.exists(crops_zip) and os.path.exists(deconv_zip):
#                             #     print(f"Profile {profile_name} already processed. Skipping.")
#                             #     continue

#                             # Prepare EcoTaxa metadata
#                             df_ET = ap.rename_for_ecotaxa(df, sample_profile_id=profile).copy()
#                             if not df_ET.empty:
#                                 ap.create_ecotaxa_zips(output_folder=results_folder, df=df_ET, profile_name=profile, 
#                                                     max_zip_size_mb=500, compression_ratio=1.0, copy_images=True, 
#                                                     add_scale_bar_to_deconv=False, pixel_resolution=23, scale_length_mm=1)
#                             else:
#                                 print(f"No data to process for profile {profile}")

#                             #ap.create_ecotaxa_zips(output_folder=results_folder, df=df_ET, profile_name=profile, max_zip_size_mb=500, compression_ratio=1.0, copy_images=True, add_scale_bar_to_deconv=False, pixel_resolution=23, scale_length_mm=1)

#                             # Save the DataFrame to a CSV file
#                             df.to_csv(os.path.join(results_folder, f'{profile}_crops_metadata.csv'), index=False)
#################################

### 3. Upload to EcoTaxa

Setup of pyecotaxa 05.06.2025: 
1. Install pyecotaxa dev_branch: git clone https://github.com/moi90/pyecotaxa.git > cd pyecotaxa > git checkout wip-aqqua > pip install .
Now pyecotaxa is useable in cli and python scripts.
2. In home directory edit the file ~/.pyecotaxa.json: {"api_token": "...", "ftp_user": "ftp_plankton", "ftp_passwd": "Pl@nkt0n4Ecotaxa"}
3. Now ftp transfer is useable.

In [6]:
import logging
import os
from pyecotaxa.remote import Remote, Transport

def login_to_ecotaxa(username, password):
    try:
        remote = Remote()
        remote.login(username, password)
        print("Successfully logged into EcoTaxa.")
        return remote
    except Exception as e:
        logging.error(f"Error during login: {e}")
        return None

def ET_upload(remote, project_id, folder_path):
    try:
        remote.push([(folder_path, project_id)],transport=Transport.FTP)
        print(f"Successfully uploaded {folder_path} to project {project_id}.")
    except Exception as e:
        logging.error(f"Error during upload: {e}") 

def upload_ecotaxa_data(profile_path, remote, project_id, crops_type='raw'):
    print(f"Processing EcoTaxa folder: {profile_path}")                
    zip_files = [f for f in os.listdir(profile_path) if f.endswith('.zip')]
    deconv_zip_files = [f for f in zip_files if 'deconv_crops' in f]
    raw_zip_files = [f for f in zip_files if 'crops' in f and 'deconv_crops' not in f]
    if crops_type == 'raw':
        # Check if there are zip files with "part" in their names
        part_zip_files = [f for f in raw_zip_files if "part" in f]

        if part_zip_files:
            # If "part" zip files exist, upload only those
            for part_zip in part_zip_files:
                zip_path = os.path.join(profile_path, part_zip)
                ET_upload(remote, project_id, zip_path)
                print(f"Uploaded {part_zip} to project {project_id}.")
        else:
            # Otherwise, upload the default crops.zip and deconv_crops.zip        
            zip_path_raw = os.path.join(profile_path, "crops.zip")
            ET_upload(remote, project_id, zip_path_raw)
            print(f"Uploaded crops.zip to project {project_id}.")
    else:
        # Upload deconv_crops.zip
        if not deconv_zip_files:
            logging.warning(f"No deconv_crops.zip found in {profile_path}.")
            return
        part_zip_files = [f for f in deconv_zip_files if "part" in f]

        if part_zip_files:
            # If "part" zip files exist, upload only those
            for part_zip in part_zip_files:
                zip_path = os.path.join(profile_path, part_zip)
                ET_upload(remote, project_id, zip_path)
                print(f"Uploaded {part_zip} to project {project_id}.")
        else:
        # Otherwise, upload the default deconv_crops.zip
            zip_path_deconv = os.path.join(profile_path, "deconv_crops.zip")
            ET_upload(remote, project_id, zip_path_deconv)
            print(f"Uploaded deconv_crops.zip to project {project_id}.")

USERNAME = 'vdausmann@geomar.de'  # Replace with your EcoTaxa username
PASSWORD = ''  # Replace with your EcoTaxa password
# Login to EcoTaxa
remote = login_to_ecotaxa(USERNAME, PASSWORD)

for profile in os.listdir(cruise_base):
    print(profile)
    profile_path = os.path.join(intermediate_result_dir, profile, profile+"_Results", "EcoTaxa")
    if not os.path.exists(profile_path):
        print(f"EcoTaxa folder does not exist for profile {profile}. Skipping.")
        continue

    if remote:
        upload_ecotaxa_data(profile_path, remote, project_id=17486, crops_type='raw')
    else:
        print("Failed to log into EcoTaxa. Exiting...")

## Upload EcoTaxa data for each profile KOSMOS PERU 2020
# for profile in os.listdir(cruise_base):
#     print(profile)
#     profile_path = os.path.join(cruise_base, profile)
#     if os.path.isdir(profile_path):                
#             for mesokosm in os.listdir(profile_path):
#                 mesokosm_path = os.path.join(profile_path, mesokosm, mesokosm+"_Results", "EcoTaxa")
#                 if not os.path.exists(os.path.join(profile_path, mesokosm, mesokosm+"_Results", "EcoTaxa")):
#                     continue
#                 print(mesokosm)
#                 if remote:
#                     upload_ecotaxa_data(mesokosm_path, remote, project_id=17486, crops_type='deconv')
#                 else:
#                     print("Failed to log into EcoTaxa. Exiting...")

Successfully logged into EcoTaxa.
SO308_1-37-4_PISCO2_20241207-1021
Processing EcoTaxa folder: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-37-4_PISCO2_20241207-1021/SO308_1-37-4_PISCO2_20241207-1021_Results/EcoTaxa


Uploading crops.zip...: 100%|██████████| 304M/304M [00:07<00:00, 40.4MB/s]
Importing to 17486... (Processing files 33400/33474): 100%|██████████| 100/100 [04:17<00:00,  2.58s/%]
100%|██████████| 1/1 [04:27<00:00, 267.73s/it]


Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-37-4_PISCO2_20241207-1021/SO308_1-37-4_PISCO2_20241207-1021_Results/EcoTaxa/crops.zip to project 17486.
Uploaded crops.zip to project 17486.
SO308_1-28-1_PISCO2_20241124-2155
Processing EcoTaxa folder: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-28-1_PISCO2_20241124-2155/SO308_1-28-1_PISCO2_20241124-2155_Results/EcoTaxa


Uploading crops.zip...: 100%|██████████| 60.1M/60.1M [00:02<00:00, 21.9MB/s]
Importing to 17486... (Processing files 8700/9396):  94%|█████████▍| 94/100 [01:11<00:04,  1.32%/s]
100%|██████████| 1/1 [01:15<00:00, 75.10s/it]


Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-28-1_PISCO2_20241124-2155/SO308_1-28-1_PISCO2_20241124-2155_Results/EcoTaxa/crops.zip to project 17486.
Uploaded crops.zip to project 17486.
SO308_1-38-3_PISCO2_20241208-0748
Processing EcoTaxa folder: /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-38-3_PISCO2_20241208-0748/SO308_1-38-3_PISCO2_20241208-0748_Results/EcoTaxa


...

Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-3-4_PISCO2_20241105-1610/SO308_1-3-4_PISCO2_20241105-1610_Results/EcoTaxa/crops_part3_upload.zip to project 17486.
Uploaded crops_part3_upload.zip to project 17486.


Uploading crops_part1_upload.zip...: 100%|██████████| 528M/528M [00:21<00:00, 25.5MB/s]
Importing to 17486... (Processing files 78900/78964): 100%|██████████| 100/100 [09:46<00:00,  5.86s/%]
100%|██████████| 1/1 [10:11<00:00, 611.36s/it]


Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-3-4_PISCO2_20241105-1610/SO308_1-3-4_PISCO2_20241105-1610_Results/EcoTaxa/crops_part1_upload.zip to project 17486.
Uploaded crops_part1_upload.zip to project 17486.


Uploading crops_part2_upload.zip...: 100%|██████████| 525M/525M [00:11<00:00, 45.9MB/s]
Importing to 17486... (Processing files 68400/68424): 100%|██████████| 100/100 [08:58<00:00,  5.39s/%]
100%|██████████| 1/1 [09:13<00:00, 553.48s/it]

Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/SO308/SO308-PISCO-Profiles/TempResults/SO308_1-3-4_PISCO2_20241105-1610/SO308_1-3-4_PISCO2_20241105-1610_Results/EcoTaxa/crops_part2_upload.zip to project 17486.
Uploaded crops_part2_upload.zip to project 17486.
